In [1]:
import polars as pl
import numpy as np
import time
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, balanced_accuracy_score
from sklearn.model_selection import KFold
from imblearn.under_sampling import RandomUnderSampler

In [2]:
df_polars_raiz = pl.read_parquet('../dataset.parquet')

In [3]:
df_polars = df_polars_raiz.sample(fraction=0.01, seed=42)

In [4]:
import ipaddress

def ip_to_int(ip: str) -> int:
    try:
        return int(ipaddress.ip_address(ip))  # Funciona tanto para IPv4 quanto IPv6
    except ValueError:
        return None

In [5]:
#df_polars = df_polars.with_columns([
#    pl.col('id.resp_h').map_elements(ip_to_int).alias('id.resp_h'),
#    pl.col('id.orig_h').map_elements(ip_to_int).alias('id.orig_h')
#])

In [6]:
df_polars = df_polars.with_columns([
    pl.col('duration').fill_null(0),
    pl.col('orig_bytes').fill_null(0),
    pl.col('resp_bytes').fill_null(0),
])

In [7]:
lista_colunas = df_polars.columns
colunas_para_spearman = ['id.resp_p', 'history', 'conn_state', 'id.orig_p', 'orig_ip_bytes', 'label']             
#['detailed-label', 'id.resp_p', 'history', 'id.orig_h', 'conn_state', 'id.orig_p', 'orig_ip_bytes']
colunas_para_dropar = [col for col in lista_colunas if col not in colunas_para_spearman]
df_polars = df_polars.drop(colunas_para_dropar)

In [8]:
df_polars

id.orig_p,id.resp_p,conn_state,history,orig_ip_bytes,label
i32,i32,i64,i64,i64,i32
5526,37215,0,0,40,1
60403,23,2,7,40,1
13386,81,2,7,40,1
36097,37215,0,0,40,1
36097,37215,0,0,40,1
…,…,…,…,…,…
30535,8081,1,1,80,1
36097,37215,0,0,40,1
41258,23,1,1,120,1


In [9]:
df_polars = df_polars.drop_nulls()

In [10]:
X = df_polars.drop('label')
y = df_polars['label']       

In [11]:
#X = X.to_numpy()
#X[:, 0] = np.array([ip_to_int(ip) for ip in X[:, 0]])

# Treinamento

In [12]:
results = []

In [13]:
def startTrain(X, y, bayes):
    kfold = KFold(n_splits=5, shuffle=True)
    results_fold = []

    for train_idx, test_idx in kfold.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Aplicando a normalização APENAS nos dados de treino
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Aplicando o undersampling no conjunto de treino
        #rus = RandomUnderSampler()
        #X_train_resampled, y_train_resampled = rus.fit_resample(X_train_scaled, y_train)
        
        start_training = time.time()
        # Definir o modelo com os parâmetros fornecidos
        bayes.fit(X_train_scaled, y_train)
        end_training = time.time()

        # Fazer previsões
        y_pred = bayes.predict(X_test_scaled)
        evaluation_time = time.time()
        #print("Predição Concluída - ", evaluation_time)        
        training_duration = end_training - start_training
        evaluation_duration = evaluation_time - end_training
        
        confusion = confusion_matrix(y_test, y_pred)
        tn, fp, fn, tp = confusion.ravel()
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        specificity = tn / (tn + fp)
        f1 = f1_score(y_test, y_pred)
        balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
        false_alarm_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
        
        avaliacao = [accuracy, balanced_accuracy, precision, recall, specificity, f1, false_alarm_rate, tn, fp, fn, tp, training_duration, evaluation_duration]
        #print(avaliacao)
        results_fold.append(avaliacao)

    results_fold_array = np.array(results_fold, dtype=np.float32)
    mean_results = np.mean(results_fold_array, axis=0)
    results.append(["Bayes"] + mean_results.tolist())

In [14]:
#Modelo: BernoulliNB - Parâmetros: {'alpha': 0.1, 'binarize': 0.1}  

In [15]:
for i in range(1,11):
    model = BernoulliNB(alpha=0.1, binarize=0.1)
    startTrain(X,y, bayes=model)
    print(i)

1
2
3
4
5
6
7
8
9
10


In [16]:
metrics_df = pl.DataFrame(
    results,
    schema=['Algorithm', 'Accuracy', 'Balanced Accuracy', 'Precision', 'Recall', 'Specificity', 'F1-score', 'False Alarm Rate', 'tn', 'fp', 'fn', 'tp', 'training_duration', 'evaluation_duration']
)
metrics_df

c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\functools.py:888: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return dispatch(args[0].__class__)(*args, **kw)


Algorithm,Accuracy,Balanced Accuracy,Precision,Recall,Specificity,F1-score,False Alarm Rate,tn,fp,fn,tp,training_duration,evaluation_duration
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Bayes""",0.981808,0.966419,0.989263,0.989069,0.943768,0.989166,0.056232,16802.199219,1001.0,1019.200012,92224.203125,0.091163,0.011822
"""Bayes""",0.981808,0.96642,0.989263,0.98907,0.943771,0.989166,0.056229,16802.199219,1001.0,1019.200012,92224.203125,0.076709,0.010704
"""Bayes""",0.981806,0.966425,0.989262,0.989067,0.943783,0.989165,0.056217,16802.199219,1001.0,1019.400024,92224.0,0.070224,0.009977
"""Bayes""",0.981806,0.966422,0.989263,0.989067,0.943777,0.989165,0.056223,16802.199219,1001.0,1019.400024,92224.0,0.067766,0.00959
"""Bayes""",0.981813,0.966427,0.989263,0.989076,0.943778,0.989169,0.056222,16802.199219,1001.0,1018.599976,92224.796875,0.065616,0.009206
"""Bayes""",0.981806,0.96642,0.989262,0.989067,0.943773,0.989165,0.056227,16802.199219,1001.0,1019.400024,92224.0,0.067663,0.00907
"""Bayes""",0.981808,0.966421,0.989262,0.989069,0.943773,0.989166,0.056227,16802.199219,1001.0,1019.200012,92224.203125,0.070088,0.010798
"""Bayes""",0.981811,0.966421,0.989263,0.989074,0.943768,0.989168,0.056232,16802.199219,1001.0,1018.799988,92224.601562,0.070763,0.009933
"""Bayes""",0.981806,0.966416,0.989263,0.989067,0.943766,0.989165,0.056234,16802.199219,1001.0,1019.400024,92224.0,0.070678,0.010225


In [17]:
metrics_df.write_csv("metrics_results/unbalanced_bayes_metrics_output.csv", separator=';')